In [ ]:
import MDAnalysis as mda
from pathlib import Path
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import matplotlib.animation as animation
import nglview as nv

In [ ]:
fpath = Path("/Users/ezry/Projects/ELIC_DPPC/DPPC/")
xtc = str(fpath.joinpath("wholereduced.xtc"))
gro = str(fpath.joinpath("md.gro"))

In [ ]:
membrane = mda.Universe(gro, xtc)
all_dppc = membrane.select_atoms('resname DPPC')
upper = all_dppc.select_atoms('prop resid <= 1521')
lower = all_dppc.select_atoms('prop resid > 1521')
water = membrane.select_atoms('resname W')
assert len(upper)==len(lower), "Warning: asymmetric membrane detected"


In [ ]:
def get_pbc(water):
    box_dims = water.bbox()
    xmin, ymin = box_dims[0,0:2]
    xmax, ymax = box_dims[1,0:2]
    xsize = xmax-xmin
    ysize = ymax-ymin

    return xsize, ysize

In [ ]:
def shift_and_wrap(positions, max):
    min = np.min(positions)
    shifted = positions-min
    wrapped = shifted%max
    return wrapped



In [ ]:
def get_counts(membrane, width):
    simlength = membrane.trajectory.n_frames-1
    xsize, ysize = get_pbc(water)
    nxs = int(xsize/width)
    nys = nxs
    all_counts = np.zeros((simlength, nxs, nys))
    for i in range(simlength):
        membrane.trajectory.next()
        xsize, ysize = get_pbc(water)
        pos = upper.positions
        initxs = pos[:,0]
        pos[:,0] = shift_and_wrap(initxs, xsize)
        pos[:,1] = shift_and_wrap(pos[:,1], ysize)
        cts, xs, ys = np.histogram2d(pos[:,0], pos[:,1], [nxs, nys])
        all_counts[i] = cts

    return all_counts

In [ ]:
def animate_counts(all_counts, fname):
    fig, ax = plt.subplots()

    ims = []
    for counts in all_counts:
        im = ax.imshow(counts, animated=True)
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                    repeat_delay=1000)

    ani.save(fname)

In [ ]:
for width in np.sqrt(np.arange(20,151,10)):
    membrane.trajectory.rewind()
    all_counts = get_counts(membrane, width)
    area = int(width**2)
    np.savetxt(f"counts_{area}.dat",all_counts.flatten())
    #animate_counts(all_counts, f"counts_{area}.mp4")